In [15]:
import time

import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument('--dns-prefetch-disable')
#driver = Chrome(chrome_options=options)

driver = webdriver.Chrome(chrome_options=chrome_options)  # Optional argument, if not specified will search path.
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
chrome_options = Options()
chrome_options.add_argument('--dns-prefetch-disable')
#driver = Chrome(chrome_options=options)
base_page = 'http://dww2.tceq.texas.gov/DWW/'
houst_systems = pd.read_csv('../Input/epa_sdwis/hous_msa_sdwis_summary.csv')
driver.get(base_page)
empty_buy = []
empty_sell = []
select_type= Select(driver.find_element_by_css_selector('#WaterSystemType'))
select_type.select_by_value('C')

count = 0
for pws_id in houst_systems['PWS ID']:
    print(pws_id)
    inputElement = driver.find_element_by_css_selector('#number')
    inputElement.clear()
    inputElement.send_keys(pws_id)
    driver.find_elements_by_css_selector('#AutoNumber1 table p input:nth-child(1)')[0].click()
    systems = driver.find_elements_by_css_selector('#AutoNumber7 td:nth-child(1) a')
    if len(systems)>0:
        systems[0].click()
        buy = driver.find_elements_by_css_selector('table:nth-child(18) td')
        sell = driver.find_elements_by_css_selector('table:nth-child(21) td')
        #sell = driver.find_elements_by_css_selector('table:nth-child(21) :nth-child(1)')
        for b in buy:
            empty_buy.append(b.text.split('\n'))
        for s in sell:
            empty_sell.append(s.text.split('\n'))
        time.sleep(.2)
        driver.back()
    driver.back()

TX1010792
TX1013205
TX1700344
TX0200519
TX1013239
TX1700427
TX1700074
TX1011459
TX1700744
TX1700798
TX1700180
TX1010145
TX1010322
TX0840009
TX1460181
TX1010684
TX1011929
TX0790189
TX1010196
TX1700651
TX1700641
TX1700287
TX1700279
TX1011803
TX1700264
TX1011687
TX0200153
TX1700529
TX1700216
TX0200190
TX0200189
TX0200242
TX0200229
TX0200227
TX0200226
TX1460145
TX1013103
TX1013127
TX1013050
TX1013089
TX1700601
TX1011860
TX1011510
TX1460102
TX1460100
TX1460096
TX1700853
TX1700852
TX1700061
TX0200223
TX0200219
TX0200218
TX0200211
TX1010163
TX1010122
TX1010116
TX1010535
TX1010532
TX0200152
TX0200151
TX0200146
TX0790261
TX0360005
TX0200363
TX0200349
TX1010915
TX0200347
TX0200341
TX1012806
TX1700041
TX1700040
TX1700015
TX1700006
TX0200337
TX0200271
TX0200258
TX0200255
TX0200254
TX0200248
TX1010100
TX1010099
TX1010092
TX1010011
TX1010622
TX1010334
TX1010307
TX1010288
TX1011651
TX1011647
TX1010254
TX1010216
TX0200415
TX0200413
TX0200067
TX0200065
TX1700666
TX1011014
TX2370049
TX0200183
TX1013011


In [89]:
empty_sell_data = pd.DataFrame()
for item in empty_sell:
    if item[0] == 'No Buyers':
        continue
    if item[0] == '':
        continue
    seller = []
    buyer = []
    pop = []
    connection = []
    empty_frame = pd.DataFrame()
    for trans in item:
        seller.append(re.sub(' ','',re.search('^TX[0-9]{,7}',trans).group(0)))
        buyer.append(re.sub(' ','',re.search(' TX[0-9]{,7}',trans).group(0)))
        pop.append(re.sub(' ','',re.split("\/", trans)[1]))
        connection.append(re.search('[A-Z]$',trans).group(0))
    empty_frame['seller'] = seller
    empty_frame['buyer'] = buyer
    empty_frame['pop'] = pop
    empty_frame['connection'] = connection
    empty_sell_data = empty_sell_data.append(empty_frame)
    #empty_sell[11]

AttributeError: 'NoneType' object has no attribute 'group'

In [92]:
empty_sell_data.to_csv('../Input/misc/physical_connections.csv')

In [140]:
empty_buy_data = pd.DataFrame()
for item in empty_buy:
    if item[0] == 'No Water Purchases':
        continue
    if item[0] == '':
        continue
    #print(item)
    seller = []
    buyer = []
    water = []
    empty_frame = pd.DataFrame()
    for trans in item:
        #print(trans)
        buyer.append(re.search('^TX[0-9]{,7}|^[0-9]',trans).group(0))
        seller.append(re.sub(' ','',re.search(' TX[0-9]{,7}',trans).group(0)))
        water.append(re.sub('^ ','',re.split("\/", trans)[1]))
    empty_frame['seller'] = seller
    empty_frame['buyer'] = buyer
    empty_frame['water'] = water
    empty_buy_data = empty_buy_data.append(empty_frame)

In [142]:
empty_buy_data.to_csv('../Input/misc/physical_connection_purchases.csv')